In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
class Video:
   
    def pozisyon_bul(self,video_path):
        
        # Video dosyasını aç
        cap = cv2.VideoCapture(video_path)

        # Fark değerlerini saklamak için bir liste oluştur
        axis_difference_list = []
        major_list = []
        minor_list = []

        while cap.isOpened():
            # Bir kareyi oku
            ret, frame = cap.read()

            # Eğer video bitmişse veya okuma başarısız olduysa döngüyü sonlandır
            if not ret:
                break

            # Görüntüyü gri tonlamaya çevir
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Gürültüyü azaltmak için bir blur işlemi uygula
            blurred = cv2.GaussianBlur(gray, (5, 5), 0)

            # Kenarları tespit et
            edges = cv2.Canny(blurred, 50, 150)

            # Contourları bul
            contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Eğer contours boşsa, devam et
            if not contours:
                continue

            # Yandan görünen tek çembere karar vermek için en büyük alanlı konturu seç
            max_contour = max(contours, key=cv2.contourArea)

            # Elips parametrelerini bul
            ellipse = cv2.fitEllipse(max_contour)
            # Elipsi çevreleyen dikdörtgenin genişliği ve yüksekliğini al
            rect_width, rect_height = ellipse[1]

            # Elipsin majör ve minör eksen uzunlukları
            major_axis = max(rect_width, rect_height) / 2
            minor_axis = min(rect_width, rect_height) / 2
            

            major_list.append(major_axis)
            minor_list.append(minor_axis)
            


            # Fark değerini hesapla ve listeye ekle
            axis_difference = abs(major_axis - minor_axis)
            axis_difference_list.append(axis_difference)
            
            if axis_difference <15:
                text = f"Fark: {axis_difference} Yola Devam Edin"
                cv2.putText(frame, text, (int(ellipse[0][0]) - 25, int(ellipse[0][1]) + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)            
            
            # Elipsi çiz
            cv2.ellipse(frame, ellipse, (0, 255, 0), 2)

            # Majör ve minör eksen uzunluklarını ekrana yazdır
            text = f"Major Eksen: {major_axis}, Minor Eksen: {minor_axis}, Fark: {axis_difference}"
            cv2.putText(frame, text, (int(ellipse[0][0]) - 50, int(ellipse[0][1]) + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)

            # Çerçeve üzerine çizilmiş görüntüyü göster
            cv2.imshow('Çember Tespiti', frame)

            # 'q' tuşuna basıldığında döngüyü sonlandır
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # Video oynatıcısını serbest bırak
        cap.release()

        # Tüm pencereleri kapat
        cv2.destroyAllWindows()
        
        return(major_list,minor_list,axis_difference_list,frame,ellipse,major_axis)
    
    
    
    
    def pozisyon_grafikleri(self,axis_difference_list,major_list,minor_list):
        if axis_difference_list:
            plt.figure(figsize=(12, 12))

            # Subplot 1
            plt.subplot(2, 2, 1)
            plt.plot(axis_difference_list, color='red')
            plt.xlabel('Çerçeve Numarası')
            plt.ylabel('Major-Minor Eksen Farkının Mutlak Değeri')
            plt.title('Major-Minor Eksen Farkının Mutlak Değerinin Değişimi')
            plt.grid(True, which='both', linestyle='--', linewidth=0.5)  # Her iki çizgiyi de göster, kesik çizgi kullan
            plt.ylim(0, max(axis_difference_list))  # Y ekseninin görüntüleme aralığını belirle

            # Subplot 2
            plt.subplot(2, 2, 2)
            plt.plot(major_list, color='blue')
            plt.xlabel('Çerçeve Numarası')
            plt.ylabel('Major Eksen')
            plt.title('Major Eksen Grafiği')
            plt.grid(True, which='both', linestyle='--', linewidth=0.5)
            plt.ylim(0, max(major_list))

            # Subplot 3
            plt.subplot(2, 2, 3)
            plt.plot(minor_list, color='yellow')
            plt.xlabel('Çerçeve Numarası')
            plt.ylabel('Minor Eksen')
            plt.title('Minor Eksen Grafiği')
            plt.grid(True, which='both', linestyle='--', linewidth=0.5)
            plt.ylim(0, max(minor_list))

            plt.tight_layout()  # Alt grafikler arasındaki boşluğu ayarla
            plt.show()
            print(np.median(axis_difference_list))
        else:
            print("Contours listesi boş. Çember bulunamadı.")
            
    
    
    
    
    def hareket(self, axis_difference_list, frame, ellipse,major_axis):
        for axis_difference in axis_difference_list:
            if axis_difference < 15:
                text = f"Fark: {axis_difference} Düz Devam Edin"
                cv2.putText(frame, text, (int(ellipse[0][0]) - 25, int(ellipse[0][1]) + 15), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 6)

            else:
                text = "Yola Devam İlerden Sol"
                cv2.putText(frame, text, (int(ellipse[0][0]) - 25, int(ellipse[0][1]) + 15), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 6)
                
        
            


In [ ]:
video_path = 'video3.mp4'  
video = Video()
major_list, minor_list, axis_difference_list,frame,ellipse,major_axis = video.pozisyon_bul(video_path)
video.hareket(axis_difference_list,frame,ellipse,major_axis)
video.pozisyon_grafikleri(axis_difference_list, major_list, minor_list);     